In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from tqdm import tqdm
import warnings
from datetime import datetime, timedelta
import statsmodels.api as sm
import os
warnings.filterwarnings("ignore")
os.getcwd()

'/Users/test/risksoft/TS/db_py'

In [2]:
try:
    os.chdir("../test123")
except FileNotFoundError:
    os.makedirs("../test123")
    os.chdir("../test123")

In [182]:
directory = 'tw/price'
list_ = os.listdir(directory)
df_list = [pd.read_parquet(os.path.join(directory, i)) for i in list_]
df_all = pd.concat(df_list)
df_all['da1'] = df_all['da'].apply(lambda x: f"{int(x[:3])+1911}-{x[4:6]}-{x[7:9]}")
df = df_all.drop_duplicates(subset=['ticker', 'da1'])
df['da_check'] = df['da1'].astype(str).apply(lambda x: int(x[:4]))
df_final = df[df['da_check'].isin([i for i in range(2018, 2026)])]

for values in ['cl', 'op', 'vol(volume)', 'vol(turnover)']:
    df_pivot = df_final.pivot(values=values, columns='ticker', index='da1')
    df_pivot.replace("--", None, inplace=True)
    df_pivot.replace("-", None, inplace=True)
    df_pivot.ffill(inplace=True)
    df_pivot = df_pivot.loc['2018-01-01':(datetime.now()+timedelta(days=1)).strftime("%Y-%m-%d")]
    df_pivot.sort_index(ascending=True, inplace=True)
    df_pivot = df_pivot.apply(lambda x: x.str.replace(',', '').astype(float))
    df_pivot.dropna(how='all', axis=1, inplace=True)
    df_pivot.to_parquet(f"tw/pdata/{values}.parquet")
    print(f"store pdata of values {values} to tw/pdata/{values}.parquet successful")
    df_pivot_pct = df_pivot.pct_change().dropna()
    df_pivot.to_parquet(f"tw/pdata/{values}_pct.parquet")
    print(f"store pdata of values {values} to tw/pdata/{values}_pct.parquet successful")
    


store pdata of values cl to tw/pdata/cl.parquet successful
store pdata of values cl to tw/pdata/cl_pct.parquet successful
store pdata of values op to tw/pdata/op.parquet successful
store pdata of values op to tw/pdata/op_pct.parquet successful
store pdata of values vol(volume) to tw/pdata/vol(volume).parquet successful
store pdata of values vol(volume) to tw/pdata/vol(volume)_pct.parquet successful
store pdata of values vol(turnover) to tw/pdata/vol(turnover).parquet successful
store pdata of values vol(turnover) to tw/pdata/vol(turnover)_pct.parquet successful


In [127]:
directory = 'tw/pb_ratio'
list_ = os.listdir(directory)
df_list = [pd.read_parquet(os.path.join(directory, i)) for i in list_]
df_all = pd.concat(df_list)
df_all['da'] = df_all.index
df = df_all.drop_duplicates(subset=['ticker', 'da'])
for values in ['pe', 'pb']:
    df_pivot = df.pivot(values=values, columns='ticker', index='da')

    df_pivot.replace("--", None, inplace=True)
    df_pivot.replace("-", None, inplace=True)
    df_pivot.ffill(inplace=True)
    df_pivot = df_pivot.loc['2018-01-01':(datetime.now()+timedelta(days=1)).strftime("%Y-%m-%d")]
    df_pivot.sort_index(ascending=True, inplace=True)
    df_pivot = df_pivot.apply(lambda x: x.str.replace(',', '').astype(float))
    df_pivot.dropna(how='all', axis=1, inplace=True)
    df_pivot.to_parquet(f"tw/pdata/{values}.parquet")
    print(f"store pdata of values {values} to tw/pdata/{values}.parquet successful")


store pdata of values pe to tw/pdata/pe.parquet successful
store pdata of values pb to tw/pdata/pb.parquet successful


### 富櫃50 TPEX

In [ ]:

import requests
list_list = []
for year in range(113, 102, -1):
    limit_month = 8 if year == 113 else 13
    for index, month in enumerate(range(1, limit_month)):
        month = f"0{month}" if month < 10 else month
        url_gretai50 = f"https://www.tpex.org.tw/web/stock/iNdex_info/gretai50/inxhis/rihisqry_result.php?l=zh-tw&d={year}/{month}/01&_=1720688879318"
        response = requests.get(url_gretai50)
        data = response.json()['aaData']
        list_list.append(data)
list_ = sum(list_list, [])
df = pd.DataFrame(list_)
df.to_parquet('../db/tw/ind/gretai50.parquet')

### 財報, selenium

In [ ]:
'''
cd /Users/test/risksoft/TS/db/chromedriver_mac/chromedriver-mac-x64/
chmod +x chromedriver

or

chmod +x /Users/test/risksoft/TS/db/chromedriver_mac/chromedriver-mac-x64/chromedriver

so that auto generated chromedriver can be reached and executed
'''

In [7]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# 初始化 Selenium WebDriver
# options = webdriver.ChromeOptions()
# options.add_argument("--headless")
service = Service(executable_path=obj.chromedriver_path)

In [2]:
import os
os.getcwd()

'/Users/test/risksoft/TS/db'

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import Select
import time
import pandas as pd
import os
os.chdir("../db/")

In [4]:
os.getcwd()

'/Users/test/risksoft/TS/db'

In [ ]:
def get_BS_hist(driver, ticker='2330', year=112, season=1):

    wait = WebDriverWait(driver, 20)
    select_element = wait.until(EC.presence_of_element_located((By.ID, 'isnew')))
    select = Select(select_element)
    select.select_by_visible_text('歷史資料')

    company_code_input = wait.until(EC.presence_of_element_located((By.ID, 'co_id')))
    company_code_input.send_keys(ticker)

    year_input = driver.find_element(By.ID, 'year')
    year_input.send_keys(str(year))

    select_element = wait.until(EC.presence_of_element_located((By.ID, 'season')))
    select = Select(select_element)
    select.select_by_visible_text(str(season))

    search_button = driver.find_element(By.CSS_SELECTOR, 'input[type="button"][value=" 查詢 "]')
    search_button.click()

    wait.until(EC.presence_of_element_located((By.ID, 'table01')))
    try_ = 2
    sleep_ = 2
    for i in range(try_):
        try:
            time.sleep(sleep_)
            data = []
            table = driver.find_element(By.CLASS_NAME, 'hasBorder')
            rows = table.find_elements(By.TAG_NAME, "tr")
            for row in rows:
                cols = row.find_elements(By.TAG_NAME, 'td')
                cols = [col.text for col in cols]
                data.append(cols)
                df = pd.DataFrame(data)
            break
        except:
            print(f"attemp {i}/{try_}")
    return df

def clean_BS_hist(df, year=112, season=1):
    season_dict = {
    1: '03-31',
    2: "06-30",
    3: "09-30",
    4: "12-31",
    }

    slices = {
        'df_current': slice(5, 15),
        'df_noncurrent': slice(18, 25),
        'df_total_asset': 26,
        'df_currentlia': slice(28, 36),
        'df_noncurrentlia': slice(38, 43),
        'df_total_lia': 44,
        'df_equity': slice(-5, -3)
    }

    # Create the sliced DataFrames
    df_sp = df.iloc[:, [0, 1, 2, -2, -1]]
    dfs = {name: df_sp.iloc[s] if isinstance(s, slice) else df_sp.iloc[[s], :] for name, s in slices.items()}
    df_concat = pd.concat(dfs.values())
    df_concat.insert(3, "season_da1", f"{1911+year}-{season_dict[season]}")
    df_concat.insert(4, "name2", df_concat.iloc[:, 0])
    df_concat.insert(7, "season_da2", f"{1911+year-1}-{season_dict[season]}")
    array_f = df_concat.values.reshape(int(df_concat.shape[0]*2), int(df_concat.shape[1]/2))
    return array_f

def BS(ticker='2330', year=112, season=1):
    arrs = []
    driver = webdriver.Chrome()
    url = 'https://mops.twse.com.tw/mops/web/t164sb03'
    driver.get(url)
    
    df = get_BS_hist(driver, ticker, year, season)
    arr = clean_BS_hist(df, year, season)
    arrs.append(arr)
    
BS()

In [85]:
season = 1
year = 112
driver = webdriver.Chrome()
url = 'https://mops.twse.com.tw/mops/web/t164sb03'
driver.get(url)

wait = WebDriverWait(driver, 20)

select_element = wait.until(EC.presence_of_element_located((By.ID, 'isnew')))
select = Select(select_element)
select.select_by_visible_text('歷史資料')

company_code_input = wait.until(EC.presence_of_element_located((By.ID, 'co_id')))
company_code_input.send_keys('2330')

year_input = driver.find_element(By.ID, 'year')
year_input.send_keys(str(year))

select_element = wait.until(EC.presence_of_element_located((By.ID, 'season')))
select = Select(select_element)
select.select_by_visible_text(str(season))

search_button = driver.find_element(By.CSS_SELECTOR, 'input[type="button"][value=" 查詢 "]')
search_button.click()

wait.until(EC.presence_of_element_located((By.ID, 'table01')))
try_ = 3
sleep_ = 3
for i in range(try_):
    try:
        data = []
        table = driver.find_element(By.CLASS_NAME, 'hasBorder')
        rows = table.find_elements(By.TAG_NAME, "tr")
        for row in rows:
            cols = row.find_elements(By.TAG_NAME, 'td')
            cols = [col.text for col in cols]
            data.append(cols)
            df = pd.DataFrame(data)
        break
    except:
        print(f"attemp {i}/{try_}")
        time.sleep(sleep_)

attemp 0/3


In [110]:
year = 112
season = 1
ticker = 2330

season_dict = {
    1: ['03-31'],
    2: ["06-30"],
    3: ["09-30"],
    4: ["12-31"],
}

slices = {
    'df_current': slice(5, 15),
    'df_noncurrent': slice(18, 25),
    'df_total_asset': 26,
    'df_currentlia': slice(28, 36),
    'df_noncurrentlia': slice(38, 43),
    'df_total_lia': 44,
    'df_equity': slice(-5, -3)
}

# Create the sliced DataFrames
df_sp = df.iloc[:, [0, 1, 2, -2, -1]]
dfs = {name: df_sp.iloc[s] if isinstance(s, slice) else df_sp.iloc[[s], :] for name, s in slices.items()}
df_concat = pd.concat(dfs.values())
df_concat.insert(3, "season_da1", f"{1911+year}-{season_dict[season]}")
df_concat.insert(4, "name2", df_concat.iloc[:, 0])
df_concat.insert(7, "season_da2", f"{1911+year-1}-{season_dict[season]}")
array_f = df_concat.values.reshape(int(df_concat.shape[0]*2), int(df_concat.shape[1]/2))